# Auto-generate travel blogs 

In [22]:
'''Example script to generate blogging text from nomadic matt's blog.

Starting off by training a simple model with very little text
'''

from __future__ import print_function

from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.callbacks import ModelCheckpoint

import keras.backend as K
import numpy as np
import random
import sys
import io
import re

The content of all the travel blogs have been crawled and output to a text file. Load the ASCII text of the travel blogs into memory.

In [2]:
# Download over a network
# path = get_file('output.txt', origin='https://www......')
# with io.open(path, encoding='utf-8') as f:
#     text = f.read().lower()

text = ''
with open("blog.txt", 'r') as content_file:
    for line in content_file.readlines():
        text += line

## Data preprocessing

In [3]:
def print_stats(text, chars):
    print('Total raw text chars:', len(text))    
    print('total vocab:', len(chars))
    print(chars)
    
chars = sorted(list(set(text)))
print_stats(text, chars)

Total raw text chars: 6654892
total vocab: 155
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '¡', '£', '®', '°', '±', '´', '¼', '½', 'Á', 'Å', 'Ç', 'É', 'Í', 'Î', '×', 'ß', 'à', 'á', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ê', 'ì', 'í', 'ï', 'ð', 'ñ', 'ò', 'ó', 'ô', 'ö', 'ø', 'ú', 'û', 'ü', 'ý', 'ā', 'ī', 'ı', 'ş', 'š', '́', '–', '—', '‘', '’', '“', '”', '…', '″', '€', '✓', '😉', '🙁', '🙂']


The total vocab for this text comes to about 155 unique characters. Most of them are not required and can be removed. We shall cleanup the text to retain only ASCII characters and convert all alphabets to lowercase, which reduces the text size and brings down the vocab count from 155 to 69.

In [4]:
def get_ASCII_text(text):
    return re.sub(r'[^\x00-\x7F]+','', text)

text = get_ASCII_text(text).lower()
chars = sorted(list(set(text)))
print_stats(text, chars)

Total raw text chars: 6614263
total vocab: 69
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~']


You can see that there are some more characters that we could remove to further clean up the dataset that will reduce the vocabulary and may improve the modeling process.

In [5]:
chars_to_remove = ['&', '[', ']', '\\', '^', '_', '`', '{', '}', '~', "'", '/']

for ch in chars_to_remove:
    if ch in text:
        text = text.replace(ch, '')
        
chars = sorted(list(set(text)))
print_stats(text, chars)

Total raw text chars: 6510374
total vocab: 58
['\n', ' ', '!', '"', '#', '$', '%', '(', ')', '*', '+', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|']


To prepare data for modelling by the neural network, we cannot model the characters directly. We should first convert the characters to integers. This can be done by mapping every character to an integer. 

Also, when preparing the mapping of unique characters to integers, we must also create a reverse mapping that we can use to convert the integers back to characters so that we can understand the predictions.

In [6]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

We now need to define the training data for the network. There is a lot of flexibility in how we choose to break up the text and expose it to the network during training.
We shall split the blog text into subsequences of 40 characters. We shall slide this window of 100 characters along the entire blog text, jumping 3 characters at every stride.

In [7]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 2170112


## One-hot encoding

First we must transform the list of input sequences into the form [samples, time steps, features] expected by an LSTM network. Next we need to rescale the integers to the range 0-to-1 to make the patterns easier to learn by the LSTM network that uses the sigmoid activation function by default. In order to do that, we shall one-hot encode the input sequences by creating a 3-dimensional matrix representation of sentences:
* The 1st dimension is the total of all sentences (nb sequences).
* The 2nd dimension is the length of each sentence, in our case 40
* The 3rd dimension is the length of total vocab/unique characters, in our case 59.

The output consists of a one-hot encoded 2-dimensional matrix. 
* The 1st dimension remains the same as input.
* The 2nd dimension has the length of total vocab/unique characters.

The output pair tells the next char for every input pair.

In [8]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
print("Total patterns", len(x))
print("Input vector shape", x.shape)
print("Output vector shape", y.shape)

Vectorization...
Total patterns 2170112
Input vector shape (2170112, 40, 58)
Output vector shape (2170112, 58)


## Building and running the model

In [9]:
# build the model: a single LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

In [10]:
def perplexity(y_true, y_pred):
    cross_entropy = K.categorical_crossentropy(y_true, y_pred)
    perplexity = K.pow(2.0, cross_entropy)
    return perplexity

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=[perplexity])

In [11]:
# Add a model checkpoint
filepath="weights-improvement.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

### Generating Text with an LSTM Network

The simplest way to use the Keras LSTM model to make predictions is to first start off with a seed sequence as input, generate the next character then update the seed sequence to add the generated character on the end and trim off the first character. This process is repeated for as long as we want to predict new characters (e.g. a sequence of 400 characters in length).

We can pick a random input pattern as our seed sequence, then print generated characters as we generate them.

In [12]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [13]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [14]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

history = model.fit(x, y,
          batch_size=128,
          epochs=1, 
          shuffle=False,
          callbacks=[print_callback, checkpoint])

Epoch 1/1
2169984/2170112 [============================>.] - ETA: 0s - loss: 1.5983 - perplexity: 15683.8785
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "red while i was there. however, even thr"
red while i was there. however, even three and the street and a be and the many of the read and i had to be a complete in the post to the signific and have to be a street and in a comment of the stranger and the street and i had to the time of an internation and when i had the stranger and the time to the travel than a coupling and the way to be the streets of the beach to the world of the street of the side of the one of the country an
----- diversity: 0.5
----- Generating with seed: "red while i was there. however, even thr"
red while i was there. however, even three way, the end to travel, i was than i had had a shoulder and day of the reading a feel that and it was a need to save an example down one of the decided to be completely met to save to be